In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import json
import os
from time import sleep
import matplotlib.pyplot as plt
import re
import string
import lxml

In [2]:
df = pd.read_csv('mojoScrape.csv')
half = df.head(5000)
small = df.head(3)

In [22]:
def getHtml(tail,tmdbId):
    top = 'http://www.boxofficemojo.com'
    t= str(tmdbId)
    buildURL = top + tail
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(buildURL, headers = header)
    sleep(.5)
    status = response.status_code
    with (open('htmlThirdTry/{}.html'.format(t),'w'))as savefile:
        savefile.write(response.text)
    return status


def checkPathOpen(tail):
    if os.path.isfile('searchHtml/{}'.format(tail))==False:
        getHtml(tail)
        sleep(.5)
    with (open('searchHtml/{}'.format(tail),'r')) as infile:
        soup = BeautifulSoup(infile,'lxml')
        return soup
    
    

s = re.compile('\ ')
punc = re.compile('[/\.]') 


def getHTMLsearch(title):
    top = 'http://www.boxofficemojo.com/search/?q='
    t = title.strip()
    fn = re.sub(s,'',t)
    p = re.sub(s,'%20',t)
    q = re.sub(punc,'',fn)
    buildURL = top+p
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(buildURL, headers = header)
    sleep(.5)
    status = response.status_code
    with (open('searchHtml/{}.html'.format(q),'w'))as savefile:
        savefile.write(response.text)
        x = savefile.seek(0,os.SEEK_END)
        return (status, x)
    
    
def load(title):
    t = title.strip()
    fn = re.sub(s,'',t)
    q = re.sub(punc,'',fn)
    return q+'.html'
    
def getSize(file):
    q = 'searchHtml/{}'.format(file)
    st = os.stat(q)
    return st.st_size

def findHref(file,title):
    soup = checkPathOpen(file)
    obj = soup.findAll('a',href=True, text ='{}'.format(title))
    if not obj:
        return np.nan
    else:
        for link in soup.findAll('a',href=True, text ='{}'.format(title)):
            if link['href']:
                return link['href']
            else:
                return np.nan
    

In [6]:
# df['status'] = df['title'].map(lambda x: getHTMLsearch(x))
df['file'] = df['title'].map(lambda x: load(x))
df.head()
# half['status'] = half['title'].map(lambda x: getHTMLsearch(x))
# small['status'] = small['title'].map(lambda x: getHTMLsearch(x))



,movieId,title,tmdbId,flatTitle,file
0,49910,Freedom Writers,1646,freedomwriters.htm,FreedomWriters.html
1,50149,Happily N'Ever After,5393,happilyneverafter.htm,HappilyN'EverAfter.html
2,50151,Home of the Brave,14171,homeofthebrave.htm,HomeoftheBrave.html
3,50153,Code Name: The Cleaner,14396,codenamethecleaner.htm,CodeName:TheCleaner.html
4,50158,Stomp the Yard,1931,stomptheyard.htm,StomptheYard.html


In [7]:
df2 = pd.read_csv('goodHTMLsea.csv')
df2.columns = ['drop', 'file']
df2.drop('drop', axis=1,inplace=True)


df2.head()

,file
0,$999.html
1,(500)DaysofSummer.html
2,(Untitled).html
3,+1.html
4,009Re:Cyborg.html


In [8]:
df2['size'] = df2['file'].map(lambda x: getSize(x))
df2 = df2[df2['size']>14000]
df2.shape

df3 = pd.merge(df2, df, how='left', on='file')



In [9]:
df3['tail'] = df3.apply(lambda row: findHref(row['file'],row['title']),axis=1)


In [25]:
# df3.dropna(inplace = True)
df3.to_csv('thirdTry.csv')


In [138]:
file = '10Years.html'
title = '10 Years'
soup = checkPathOpen(file)
matchstr = re.compile('1 Movie Matches:')
# for link in soup.find_all('a', href=True, text='10 Years'):
#     print(link)


print(findHref(file, title))

/movies/?id=10years.htm


In [18]:
df4 = df3[['title', 'tmdbId','tail']]
df4.head()

,title,tmdbId,tail
0,$9.99,19311,/movies/?id=9dot99.htm
3,10 Years,58547,/movies/?id=10years.htm
5,12,20714,/movies/?id=12.htm
7,127 Hours,44115,/movies/?id=127hours.htm
8,12 Rounds,17134,/movies/?id=12rounds.htm


In [23]:
df4['status'] = df4.apply(lambda row: getHtml(row['tail'], row['tmdbId']),axis = 1)
df4.head()

/Users/kylemix/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,title,tmdbId,tail,status
0,$9.99,19311,/movies/?id=9dot99.htm,200
3,10 Years,58547,/movies/?id=10years.htm,200
5,12,20714,/movies/?id=12.htm,200
7,127 Hours,44115,/movies/?id=127hours.htm,200
8,12 Rounds,17134,/movies/?id=12rounds.htm,200
